In [1]:
import os
os.chdir('../..')

In [2]:
from modules.table_evaluator import load_data, TableEvaluator
# From https://github.com/Baukebrenninkmeijer/table-evaluator

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
dataset = 'Chicago'
synth_dir = '../synth_data/' + dataset

files = ['../data/' + dataset + '/data.csv']
labels = ['original', 'WGAN_WI_SIM', 'WGAN_WI2_SIM', 'WGAN_WI_SIMCONT', 'WGAN_WI2_SIMCONT', 'WGAN_WI_ARGMAX', 'WGAN_WI2_ARGMAX']

for l in labels[1:]:
    files.append('../synth_data/' + dataset + '/' + l + '.csv')

In [4]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [5]:
dfs = [pd.read_csv(file) for file in files]

In [6]:
cat_cols = list(set(dfs[0].columns) - set(continuous_columns))

In [7]:
n=10000

In [8]:
print("random sampling:")
train = dfs[0].sample(int(len(dfs[0]) * 0.5)).sample(n)
test = dfs[0][~dfs[0].index.isin(train.index)].sample(n)
table_evaluator = TableEvaluator(train, test, cat_cols)
table_evaluator.evaluate(target_col='choice')
print('--------------------------------------------------------')
print('')

for i, l in enumerate(labels[1:]):
    print(l + ":")
    train = dfs[0].sample(n)
    test = dfs[i+1].sample(n)
    test = test[train.columns]
    table_evaluator = TableEvaluator(train, test, cat_cols)
    table_evaluator.evaluate(target_col='choice')
    print('--------------------------------------------------------')
    print('')

random sampling:

Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                             f1_real  f1_fake  jaccard_similarity
index                                                            
DecisionTreeClassifier_fake   0.6851   0.6823              0.4801
DecisionTreeClassifier_real   0.6795   0.6825              0.4752
LogisticRegression_fake       0.7524   0.7519              0.9639
LogisticRegression_real       0.7519   0.7522              0.9648
MLPClassifier_fake            0.7683   0.7628              0.8181
MLPClassifier_real            0.7660   0.7687              0.8239
RandomForestClassifier_fake   0.7826   0.7725              0.7699
RandomForestClassifier_real   0.7797   0.7790              0.7664

Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.0054
Column Correlation distance MAE          0.0035
Duplicate rows between sets (real/fake)  (4, 8)
nearest neighbor mean  


Miscellaneous results:
                                          Result
Column Correlation Distance RMSE          0.0151
Column Correlation distance MAE           0.0092
Duplicate rows between sets (real/fake)  (12, 0)
nearest neighbor mean                     1.4622
nearest neighbor std                      0.4577

Results:
                                                Result
Basic statistics                                1.0000
Correlation column correlations                 0.9967
Mean Correlation between fake and real columns  0.9293
1 - MAPE Estimator results                      0.9823
1 - MAPE 5 PCA components                       0.9676
Similarity Score                                0.9752
--------------------------------------------------------

WGAN_WI2_ARGMAX:

Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                             f1_real  f1_fake  jaccard_similarity
index                                                         

In [9]:
print("random sampling:")
train = dfs[0].sample(int(len(dfs[0]) * 0.5))
test = dfs[0][~dfs[0].index.isin(train.index)]
table_evaluator = TableEvaluator(train.sample(n), test.sample(n), cat_cols)
table_evaluator.evaluate(target_col='age', target_type='regr')
print('--------------------------------------------------------')
print('')

for i, l in enumerate(labels[1:]):
    print(l + ":")
    train = dfs[0].sample(n)
    test = dfs[i+1].sample(n)
    test = test[train.columns]
    table_evaluator = TableEvaluator(train, test, cat_cols)
    table_evaluator.evaluate(target_col='age', target_type='regr')
    print('--------------------------------------------------------')
    print('')

random sampling:

Correlation metric: pearsonr

Regressor MSE-scores and their Jaccard similarities:
                                   real    fake
fake_data_ElasticNet            15.1333 15.3378
fake_data_Lasso                 14.3806 14.5373
fake_data_RandomForestRegressor  9.8547 10.0181
fake_data_Ridge                 14.0046 14.1018
real_data_ElasticNet            15.1562 15.3612
real_data_Lasso                 14.4581 14.6050
real_data_RandomForestRegressor  9.8134 10.0665
real_data_Ridge                 14.0159 14.1017

Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.0074
Column Correlation distance MAE          0.0044
Duplicate rows between sets (real/fake)  (2, 6)
nearest neighbor mean                    1.2780
nearest neighbor std                     0.5322

Results:
                                                Result
Basic statistics                                1.0000
Correlation column correlations    


Miscellaneous results:
                                          Result
Column Correlation Distance RMSE          0.0160
Column Correlation distance MAE           0.0093
Duplicate rows between sets (real/fake)  (12, 0)
nearest neighbor mean                     1.4593
nearest neighbor std                      0.4631

Results:
                                                Result
Basic statistics                                0.9930
Correlation column correlations                 0.9961
Mean Correlation between fake and real columns  0.9421
Correlation RMSE                                0.9907
1 - MAPE 5 PCA components                       0.9482
Similarity Score                                0.9740
--------------------------------------------------------

